In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math

# Read in data:
# Cloud_Sections
cloud_sections = pd.read_csv('Cleaned/CLOUD_SECTIONS_Cleaned.csv', delimiter = ',', encoding = 'latin1')

# Designed_Pipe_Center
designed_centers = pd.read_csv('Cleaned/DESIGNED_Pipe_Centers_Cleaned.csv', delimiter = ',', encoding = 'latin1')

# Planes_Local_Global_Cleaned
p_info = pd.read_csv('Cleaned/Planes_Local_Global_Cleaned.csv', delimiter = ',', encoding = 'latin1')

In [2]:
# A List of sections, where each section is a list of tuples, and each tuple is the XY coordinates of a point on the point cloud
TuplePairsBySection = [] 

# A list of the Designed Center points and radius, each object in the list is a panda dataframe where there are as many rows as pipes
CenterList = [] 

# For n in the number of distinct sections
for n in range(cloud_sections.Section.nunique()): 
    
    # Grabs all dataframe rows in each distinct section
    grab = cloud_sections[cloud_sections['Section'] == 'Section-{' + str(n) + '}'] 
    
    # Adds the individual X and Y points in each point cloud pair for the nth section
    TuplePairsBySection.append(list(zip(grab.X, grab.Y))) 
    
    # Adds the dataframe rows in each distinct section
    CenterList.append(designed_centers[designed_centers['Section'] == 'Section-{' + str(n) + '}']) 

In [3]:
# Arbitrary radius multiplier
N = 1.4

# A List of points in the point cloud, by section and pipe, that pass the first noise test
PassesFirstNoiseTestBySection = [] 

# A List of points in the point cloud, by section, that fail the first noise test
FailsFirstNoiseTestBySection = [] 

count = 0 
for section in TuplePairsBySection:
    D, E, F = [], [], []
    
    # Intialize the radius for both pipes
    r1, r2 = CenterList[count].iloc[0,4], CenterList[count].iloc[1,4] 
    
    # Intialize the center XY coordinate tuple for both pipes
    c1, c2 = tuple(CenterList[count].iloc[0,1:3]), tuple(CenterList[count].iloc[1,1:3]) 
    
    for point in section: 
        
        # Test first pipe
        if math.dist(c1, point) < (r1 * N): 
            D.append(point)
            
        # Test second pipe
        elif math.dist(c2, point) < (r2 * N): 
            E.append(point)
        else:
            F.append(point)
            
    PassesFirstNoiseTestBySection.append(D)
    PassesFirstNoiseTestBySection.append(E)
    FailsFirstNoiseTestBySection.append(F)
    count += 1

In [4]:
# Arbitrary IQR multiplier
M = 1.3

def findOutVals(rlist):
    P, F = {}, {}
    P_index = []
    sortedList = sorted(rlist)
    L = len(rlist)
    Q1 = np.median(sortedList[:(L//2 + 2)]) 
    Q3 = np.median(sortedList[(L//2 + 1):])
    IQR = Q3 - Q1
    L_out = Q1 - IQR*M
    R_out = Q3 + IQR*M
    count = 0
    for n in rlist:
        if (n > L_out) and (n < R_out):
            P[count] = n
        count += 1
    for a in P:
        P_index.append(a)
    return P_index

In [5]:
# A List of points in the point cloud, by section and pipe, that pass the first noise test
PassesSecondNoiseTestBySection = [] 

# A List of points in the point cloud, by section, that fail the first noise test
FailsSeconNoiseTestBySection = [] 

count = 0 
for circle in PassesFirstNoiseTestBySection:
    passPoints, failPoints = [], []
    if len(circle) == 0:
        count += 1
    else:
        ResList = []
        if count %2 == 0:
            r = CenterList[count // 2].iloc[0,4]
            c = tuple(CenterList[count // 2].iloc[0,1:3])
            for p in circle:
                ResList.append((math.dist(p, c) - r))
            Pass_index = findOutVals(ResList)
            for n in range(len(circle)):
                if n in Pass_index:
                    passPoints.append(circle[n])
                else:
                    failPoints.append(circle[n])                 
        else:
            r = CenterList[count // 2].iloc[1,4]
            c = tuple(CenterList[count // 2].iloc[1,1:3])
            for p in circle:
                ResList.append((math.dist(p, c) - r))
            findOutVals(ResList)
            Pass_index = findOutVals(ResList)
            for n in range(len(circle)):
                if n in Pass_index:
                    passPoints.append(circle[n])
                else:
                    failPoints.append(circle[n])
            count += 1       
    PassesSecondNoiseTestBySection.append(passPoints)
    FailsSeconNoiseTestBySection.append(failPoints)

In [ ]:
FACECOLOR = '#fffdfd'
FIGSIZE = (36,36)
TICK_LABELSIZE = 54
TICK_LENGTH = 24
TICK_WIDTH = 4
TITLE_FONTSIZE = 58
MARKER_SIZE = 544
ANNOTATE_SIZE = 34

for NUM in range(50):
    
    # Double cleaned sections---------------------------------------------------
    
    double_clean_fig = plt.figure(figsize = FIGSIZE, facecolor = FACECOLOR)
    double_clean_section = double_clean_fig.add_subplot(1,1,1)
    double_clean_section.set_title('Cleaned Twice Section ' + str(NUM), fontsize = TITLE_FONTSIZE)
    double_clean_section.set_facecolor(FACECOLOR)
    double_clean_section.scatter(*zip(*TuplePairsBySection[NUM]), s = MARKER_SIZE)
    double_clean_section.scatter(CenterList[NUM].X, CenterList[NUM].Y, color = 'red', s = MARKER_SIZE)
    double_clean_section.annotate(str(len(TuplePairsBySection[NUM])), (40, 10), fontsize = ANNOTATE_SIZE)
    double_clean_section.tick_params(labelsize = TICK_LABELSIZE, length = TICK_LENGTH, width = TICK_WIDTH)
    
    # Section 1 circle 3 workaround
    
    if len(PassesSecondNoiseTestBySection[2 * NUM]) > 0: 
        double_clean_section.scatter(*zip(*PassesSecondNoiseTestBySection[2 * NUM]), color = 'orange', s = MARKER_SIZE) #Circle saved as D
    else: 
        print('Circle ' + str(2 * NUM) + ' from Section ' + str(NUM) + ' has no values.')

    double_clean_section.annotate(str(len(PassesSecondNoiseTestBySection[2 * NUM])) + '/' + str(len(PassesFirstNoiseTestBySection[2 * NUM])),
                 (CenterList[NUM].iloc[0,1] + 10, CenterList[NUM].iloc[0,2]), fontsize = ANNOTATE_SIZE)
    
    # Section 1 circle 3 workaround
    
    if len(PassesSecondNoiseTestBySection[2 * NUM + 1]) > 0: 
        double_clean_section.scatter(*zip(*PassesSecondNoiseTestBySection[2 * NUM + 1]), color = 'yellow', s = MARKER_SIZE)  #Circle saved as E
    else: 
        print('Circle ' + str(2 * NUM + 1) + ' from Section ' + str(NUM) + ' has no values.')

    double_clean_section.annotate(str(len(PassesSecondNoiseTestBySection[2 * NUM + 1])) + '/' + str(len(PassesFirstNoiseTestBySection[2 * NUM + 1])),
                  (CenterList[NUM].iloc[1,1] + 10, CenterList[NUM].iloc[1,2] + 10), fontsize = ANNOTATE_SIZE)
    double_clean_section.annotate(str(len(PassesSecondNoiseTestBySection[2 * NUM]) + len(PassesSecondNoiseTestBySection[2 * NUM + 1])), (40, 10), fontsize = ANNOTATE_SIZE)
    double_clean_section.scatter(CenterList[NUM].X, CenterList[NUM].Y, color = 'red', s = MARKER_SIZE)
    
    # save final figure
    # Uncomment following line when ready to save images
    
    #double_clean_fig.savefig('Results/Cleaned_Twice_Sections/cleaned_twice_section_' + str(NUM) + '.png', facecolor = double_clean_fig.get_facecolor())
    
    # Single cleaned sections---------------------------------------------------
    
    single_clean_fig = plt.figure(figsize = FIGSIZE, facecolor = FACECOLOR)
    single_clean_section = single_clean_fig.add_subplot(1,1,1)
    single_clean_section.set_title('Cleaned Once Section ' + str(NUM), fontsize = TITLE_FONTSIZE)
    single_clean_section.set_facecolor(FACECOLOR)
    single_clean_section.scatter(*zip(*TuplePairsBySection[NUM]), s = MARKER_SIZE)
    single_clean_section.scatter(CenterList[NUM].X, CenterList[NUM].Y, color = 'red', s = MARKER_SIZE)
    single_clean_section.annotate(str(len(TuplePairsBySection[NUM])), (40, 10), fontsize = ANNOTATE_SIZE)
    single_clean_section.tick_params(labelsize = TICK_LABELSIZE, length = TICK_LENGTH, width = TICK_WIDTH)
    
    # Section 1 circle 3 workaround
    
    if len(PassesSecondNoiseTestBySection[2 * NUM]) > 0: 
        single_clean_section.scatter(*zip(*PassesSecondNoiseTestBySection[2 * NUM]), color = 'orange', s = MARKER_SIZE) #Circle saved as D
    else: 
        print('Circle ' + str(2 * NUM) + ' from Section ' + str(NUM) + ' has no values.')

    single_clean_section.annotate(str(len(PassesSecondNoiseTestBySection[2 * NUM])) + '/' + str(len(PassesFirstNoiseTestBySection[2 * NUM])),
                 (CenterList[NUM].iloc[0,1] + 10, CenterList[NUM].iloc[0,2]), fontsize = ANNOTATE_SIZE)
    
    # Section 1 circle 3 workaround
    
    if len(PassesSecondNoiseTestBySection[2 * NUM + 1]) > 0: 
        single_clean_section.scatter(*zip(*PassesSecondNoiseTestBySection[2 * NUM + 1]), color = 'yellow', s = MARKER_SIZE)  #Circle saved as E
    else: 
        print('Circle ' + str(2 * NUM + 1) + ' from Section ' + str(NUM) + ' has no values.')

    single_clean_section.annotate(str(len(PassesSecondNoiseTestBySection[2 * NUM + 1])) + '/' + str(len(PassesFirstNoiseTestBySection[2 * NUM + 1])),
                  (CenterList[NUM].iloc[1,1] + 10, CenterList[NUM].iloc[1,2] + 10), fontsize = ANNOTATE_SIZE)

    single_clean_section.annotate(str(len(PassesSecondNoiseTestBySection[2 * NUM]) + len(PassesSecondNoiseTestBySection[2 * NUM + 1])), (40, 10), fontsize = ANNOTATE_SIZE)
    single_clean_section.scatter(CenterList[NUM].X, CenterList[NUM].Y, color = 'red', s = MARKER_SIZE)
    
    # save final figure
    # Uncomment following line when ready to save images
    
    #single_clean_fig.savefig('Results/Cleaned_Once_Sections/cleaned_once_section_' + str(NUM) + '.png', facecolor = single_clean_fig.get_facecolor())
    
    
    # Original Section----------------------------------------------------------
    
    original_fig = plt.figure(figsize = FIGSIZE, facecolor = FACECOLOR)
    original_section = original_fig.add_subplot(1,1,1)
    original_section.set_title('Original Section ' + str(NUM), fontsize = TITLE_FONTSIZE)
    original_section.set_facecolor(FACECOLOR)
    original_section.scatter(*zip(*TuplePairsBySection[NUM]), s = MARKER_SIZE)
    original_section.scatter(CenterList[NUM].X, CenterList[NUM].Y, color = 'red', s = MARKER_SIZE)
    original_section.annotate(str(len(TuplePairsBySection[NUM])), (40, 10), fontsize = ANNOTATE_SIZE)
    original_section.tick_params(labelsize = TICK_LABELSIZE, length = TICK_LENGTH, width = TICK_WIDTH)
    
    # save final figure
    # Uncomment following line when ready to save images
    
    #original_fig.savefig('Results/Original_Sections/original_section_' + str(NUM) + '.png', facecolor = original_fig.get_facecolor())
    
    # Incase these lines are needed 
    #plt.xlim(0, max(x[0] for x in TuplePairsBySection[NUM]) * 1.25)
    #plt.ylim(0, max(y[1] for y in TuplePairsBySection[NUM]) * 1.25)

In [7]:
print(PassesSecondNoiseTestBySection[0])

[(61.61, 59.48), (60.87, 60.03), (58.78, 62.92), (61.97, 59.21), (58.19, 64.98), (61.14, 59.77), (59.88, 60.98), (60.94, 59.96), (68.11, 58.33), (62.83, 58.74), (63.58, 58.41), (63.94, 58.34), (62.24, 59.01), (63.39, 58.53), (64.37, 58.26)]


In [7]:
def printInfo(radius, Xcp, Ycp, secNum, pipeCount):
    #round values
    roundList = [radius, Xcp, Ycp]
    roundList = [round(i, 6) for i in roundList]
    radius = roundList[0]
    Xcp = roundList[1]
    Ycp = roundList[2]
    FCP = (Xcp, Ycp)
    givenR = CenterList[secNum].iloc[pipeCount,4]
    givenC_tuple = tuple(CenterList[secNum].iloc[pipeCount,1:3])
    print("The fitted center point is: (" + str(Xcp) + "," + str(Ycp) + ")")
    print("The as-designed center point is: (" + str(givenC_tuple[0]) + "," + str(givenC_tuple[1]) + ")")
    print("The distance between them is: " + str(round(math.dist(givenC_tuple, FCP), 7)))
    print("The fitted radius is: " + str(radius) + " and the actual radius is: " + str(givenR))
    print("The difference between them is: " + str(round(abs(givenR - radius), 7)))
    


In [ ]:
from circle_fit import hyper_fit, plot_data_circle

sec_count = 0
for n in range(50): 
    a = PassesSecondNoiseTestBySection[2 * n:2 * n + 2]
    if len(a[0]) > 0:
        xc,yc,r,s = hyper_fit(a[0])
        plot_data_circle((list(zip(*a[0]))[0]),(list(zip(*a[0]))[1]),xc,yc,r)
        plt.title("Section " + str(sec_count) + " Pipe " + str(sec_count*2))
        plt.show()
        printInfo(r, xc, yc, sec_count, 0)
    else:
        pass
    if len(a[1]) > 0:
        xc,yc,r,s = hyper_fit(a[1])
        plot_data_circle((list(zip(*a[1]))[0]),(list(zip(*a[1]))[1]),xc,yc,r)
        plt.title("Section " + str(sec_count) + " Pipe " + str(sec_count*2 + 1))
        plt.show()
        printInfo(r, xc, yc, sec_count, 1)
    else:
        pass
    sec_count += 1